# 🐚 Welcome to FOSSIL!

            ..\\\\
         ..\\\\  ,\
    --===)))))))   )
         ''////  `/
            ''////  

                                                                                                                   
## Overview
This tool uses neural network templates alongside the CEGIS architecture to automatically synthesise sound Lyapunov functions of Barrier Certificates for any N-dimensional system. 

To use this tool please change the python code in the corresponding cells to input your required dynamical system and domain, then use the correspoding cell to synthise a [Lyapunov function](#lyap-synth) of [Barrier function](#barrier-synth) respectively.

If you would like access to more settings, such as , please use the [advanced-playground](FOSSIL-advanced-playground.ipynb). To see benchmarks and further examples refer to [benchmarks](FOSSIL-benchmarks.ipynb).

In [1]:
# % Imports
import sys
sys.path.append('..')
from experiments.playground_utils import *
#sys.path.append(".")

### Synthesising Lyapunov Functions

First, we demonstrate synthesising Lyapunov functions. Consider the following dynamical system:

$$ \begin{cases} \dot{x} = -x + xy \\ \dot{y} = - y \end{cases}$$


This system can be instantied as follows, with $x_0 = x $ and $x_1 = y$:


In [2]:

N_Dimensions = 2

x0, x1 = initialise_states(N_Dimensions)

dynamics = [
    -x0 + x0 * x1,
    -x1
]

# Note, any arbitrary variables can be used here, but internally they will be treated as x_0,...,x_n and the
# returned function will be a function of these parameters.

This system has an equilibrium point at the origin $(x_0, x_1) = (0, 0)$. We seek to synthesis a Lyapunov function to serve as a certificate of stability of this equilibrium point. This Lyapunov function will be validated in a region around the equilibrium point. This domain is defined as:

$$ \mathcal{D}  = \{(x_0, x_1): x_0^2 + x_1^2 < r^2 \} $$

for some user-set parameter r. To constrain the domain to a torus shape (i.e., excluding a smaller hypershere around the origin), use the keyword arguement INNER_RADIUS. 
<a id='lyap-synth'></a>

In [3]:
r = 10
mode = PrimerMode.LYAPUNOV
numerical_V, symbolic_V = synthesise(dynamics, mode, OUTER_RADIUS=r, CEGIS_PARAMETERS={CegisConfig.SEED_AND_SPEED.k: True})

Single Equilibrium point found: 
 [(0, 0)]
  learner   0
0 - loss: 15.74869155883789 - acc: 42.8 %
  regulariser   0


/home/andffo/PycharmProjects/LRNN/venv/lib/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag


Candidate: 4972663187604749/50000000000000000*x0**2 +
38026414961365913/10000000000000000*x1**2 +
455040111650719/3125000000000000*x0*x1
  verifier   0
No counterexamples found!
  trajectoriser   0


In [4]:
# The symbolic function can be printed using print_f
print_f(symbolic_V)

4972663187604749/50000000000000000*x0**2 +
38026414961365913/10000000000000000*x1**2 +
455040111650719/3125000000000000*x0*x1


### Barrier Cerificates


We now demonstrate synthesising a barrier certificate. Consider the following dynamical system:

$$\begin{cases}
\dot{x} = y \\
\dot{y} = - 2400y - 576x
\end{cases}$$

We can initialise this dynamical system as follows, with with $x_0 = x $ and $x_1 = y$.

In [5]:
N_Dimensions = 2

x0, x1= initialise_states(N_Dimensions)

dynamics = [
    x1,
    -2400*x1 - 576*x0
    
]

#### Domain Initialisation
We define the overall domain as 
$$\begin{align*}
& X = \{x_0^2 + x_1^2 \leq 4\}\\ 
& X_0 = \{(x_0 - 1)^2+(x_1 - 1)^2 \leq 0.25 \} \\ 
& X_u  = \{ (x_0 + 2)^2+(x_1 + 2)^2 \leq 0.16 \}
\end{align*}$$


This can be initialised as follows:

In [6]:
xd = Sphere([0, 0], 2)
xi = Sphere([1, 1], 0.5)
xu = Sphere([-2, -2], 0.4)

#Hyper cuboids can be generated using Rectangle([lower_bounds], [upper_bounds])

A barrier function is then synthesised as:
<a id='barrier-synth'></a>

In [7]:
verifier_type = VerifierType.DREAL
mode = PrimerMode.BARRIER

f_n, f_v, = synthesise(dynamics, mode, XD=xd, XI=xi, XU=xu, 
                       CEGIS_PARAMETERS={CegisConfig.VERIFIER.k:VerifierType.DREAL, 
                                         CegisConfig.ACTIVATION.k: [ActivationType.LINEAR] })


  learner   0
0 - loss: -2.700150966644287 - accuracy init-unsafe: 0.0 - accuracy belt: 49.05263157894737 - points in belt: 475
  regulariser   0
Candidate: (-1.8496560265007025 - 1.260275037767739 * x0 - 1.6037890409201969 * x1)
  verifier   0
No counterexamples found!
Certified!
  trajectoriser   0
Certified!
Learner times: total=0.5576460920001409s,min=0.5576460920001409s,max=0.5576460920001409s,avg=0.5576460920001409s
Regulariser times: total=0.08034187799967185s,min=0.08034187799967185s,max=0.08034187799967185s,avg=0.08034187799967185s
Verifier times: total=0.0026628159998836054s,min=0.0026628159998836054s,max=0.0026628159998836054s,avg=0.0026628159998836054s
Trajectoriser times: total=0s,min=9223372036854775808s,max=0s,avg=0s
